In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
import os, shutil, zipfile
from glob import glob
import matplotlib.pyplot as plt

In [ ]:
alexnet = models.Sequential([
    layers.Input(shape=(227, 227, 3)),
    layers.Conv2D(filters=96, kernel_size=(11, 11), strides=(4, 4), activation='relu'),
    layers.MaxPooling2D(pool_size=(3, 3), strides=(2, 2)),

    layers.Conv2D(filters=256, kernel_size=(5, 5),padding='same', strides=(1, 1), activation='relu'),
    layers.MaxPooling2D(pool_size=(3, 3), strides=(2, 2)),

    layers.Conv2D(filters=384, kernel_size=(3, 3), padding='same', strides=(1, 1), activation='relu'),
    layers.Conv2D(filters=384, kernel_size=(3, 3), padding='same', strides=(1, 1), activation='relu'),
    layers.Conv2D(filters=256, kernel_size=(3, 3), padding='same', strides=(1, 1), activation='relu'),
    layers.MaxPooling2D(pool_size=(3, 3), strides=(2, 2)),

    layers.Flatten(),
    layers.Dense(4096, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(4096, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1000, activation='softmax')
])

alexnet.summary()



Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 55, 55, 96)     │        34,944 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 27, 27, 96)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 27, 27, 256)    │       614,656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 13, 13, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 13, 13, 384)    │       885,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 13, 13, 384)    │     1,327,488 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 13, 13, 256)    │       884,992 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 6, 6, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 9216)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 4096)           │    37,752,832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 4096)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 4096)           │    16,781,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 4096)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1000)           │     4,097,000 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 62,378,344 (237.95 MB)

 Trainable params: 62,378,344 (237.95 MB)

 Non-trainable params: 0 (0.00 B)

# Implement AlexNet using Pytorch

In [1]:
from google.colab import files
files.upload()

KeyboardInterrupt: 

In [ ]:
os.makedirs('/root/.kaggle', exist_ok=True)
shutil.move('kaggle (2).json', '/root/.kaggle/kaggle.json')
os.chmod('/root/.kaggle/kaggle.json', 0o600)

In [ ]:
!kaggle datasets download -d imsparsh/flowers-dataset -p /content/flowers --unzip


Dataset URL: https://www.kaggle.com/datasets/imsparsh/flowers-dataset
License(s): CC0-1.0
 68% 139M/205M [00:00<00:00, 1.45GB/s]
100% 205M/205M [00:00<00:00, 963MB/s] 


In [ ]:
class_names = os.listdir("/content/flowers/train")
class_names

['sunflower', 'rose', 'dandelion', 'tulip', 'daisy']

In [ ]:
dir_train_path = "/content/flowers/train"

for class_name in class_names:
    print(f"{class_name}: {len(os.listdir(os.path.join(dir_train_path, class_name)))}")

sunflower: 495
rose: 497
dandelion: 646
tulip: 607
daisy: 501


# Now we use another dataset tha consist of 5000 images

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import wandb

In [3]:
wandb.init(project="AlexNet_Flower_v1", config={
    "learning_rate": 0.001,
    "epochs": 10,
    "batch_size": 32,
    "architecture": "alexnet",
    "pretrained": True,
    "input_size": 128
    })



wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice:

 1


wandb: You chose 'Create a W&B account'
wandb: Create an account here: https://wandb.ai/authorize?signup=true&ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: ERROR Invalid API key: API key may only contain the letters A-Z, digits and underscores.
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice:

...

wandb: WARNING Invalid choice
wandb: Enter your choice:

 1


wandb: You chose 'Create a W&B account'
wandb: Create an account here: https://wandb.ai/authorize?signup=true&ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: mohammadjavad-mousavi97 (mohammadjavad-mousavi97-arak-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [4]:
config = wandb.config

In [5]:
transform = transforms.Compose([
    transforms.Resize((config.input_size, config.input_size)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [6]:
train_dir = "/content/drive/MyDrive/Colab Notebooks/flowers/train"
val_dir = "/content/drive/MyDrive/Colab Notebooks/flowers/val"

train_dataset = datasets.ImageFolder(train_dir, transform=transform)
val_dataset = datasets.ImageFolder(val_dir, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=config.batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=config.batch_size, shuffle=False)

In [8]:
alexnet = models.alexnet(pretrained=config.pretrained)

for param in alexnet.parameters():
    param.requires_grad = False

alexnet.classifier[6] = nn.Linear(4096, 5)

for param in alexnet.classifier[6].parameters():
    param.requires_grad = True

alexnet = alexnet.to('cuda' if torch.cuda.is_available() else 'cpu')

wandb.watch(alexnet, log="all", log_freq=10)

In [9]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(alexnet.parameters(), lr=config.learning_rate)

In [10]:
def train_model(model, criterion, optimizer, train_loader, val_loader, num_epochs=10, log_every=10):
    device = next(model.parameters()).device

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0

        for step, (images, labels) in enumerate(train_loader):
            images = images.to(device, non_blocking=True)
            labels = labels.to(device, non_blocking=True)

            optimizer.zero_grad(set_to_none=True)
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            bs = labels.size(0)
            running_loss += loss.item() * bs
            preds = outputs.argmax(dim=1)
            correct += (preds == labels).sum().item()
            total += bs

            if (step + 1) % log_every == 0:
                train_loss_step = running_loss / max(1, total)
                train_acc_step = correct / max(1, total)
                wandb.log(
                    {
                        "epoch": epoch + 1,
                        "step": epoch * len(train_loader) + (step + 1),
                        "train/loss": train_loss_step,
                        "train/acc": train_acc_step,
                        "lr": optimizer.param_groups[0]["lr"],
                    }
                )

        train_loss = running_loss / max(1, total)
        train_acc = correct / max(1, total)

        model.eval()
        val_loss_sum = 0.0
        val_correct = 0
        val_total = 0

        with torch.no_grad():
            for images, labels in val_loader:
                images = images.to(device, non_blocking=True)
                labels = labels.to(device, non_blocking=True)

                outputs = model(images)
                loss = criterion(outputs, labels)

                bs = labels.size(0)
                val_loss_sum += loss.item() * bs
                preds = outputs.argmax(dim=1)
                val_correct += (preds == labels).sum().item()
                val_total += bs

        val_loss = val_loss_sum / max(1, val_total)
        val_acc = val_correct / max(1, val_total)

        wandb.log(
            {
                "epoch": epoch + 1,
                "train/loss_epoch": train_loss,
                "train/acc_epoch": train_acc,
                "val/loss": val_loss,
                "val/acc": val_acc,
            }
        )

        print(
            f"Epoch {epoch+1}/{num_epochs} | "
            f"train_loss={train_loss:.4f} train_acc={train_acc:.4f} | "
            f"val_loss={val_loss:.4f} val_acc={val_acc:.4f}"
        )

    return model


In [11]:
alexnet = train_model(
    model=alexnet,
    criterion=criterion,
    optimizer=optimizer,
    train_loader=train_loader,
    val_loader=val_loader,
    num_epochs=config.epochs,
    log_every=10
)

wandb.finish()


Epoch 1/10 | train_loss=0.7355 train_acc=0.7478 | val_loss=0.5489 val_acc=0.8091
Epoch 2/10 | train_loss=0.4211 train_acc=0.8451 | val_loss=0.4592 val_acc=0.8546
Epoch 3/10 | train_loss=0.3382 train_acc=0.8792 | val_loss=0.4598 val_acc=0.8477
Epoch 4/10 | train_loss=0.3076 train_acc=0.8890 | val_loss=0.3946 val_acc=0.8813
Epoch 5/10 | train_loss=0.2692 train_acc=0.9024 | val_loss=0.3818 val_acc=0.8773
Epoch 6/10 | train_loss=0.2363 train_acc=0.9134 | val_loss=0.4163 val_acc=0.8724
Epoch 7/10 | train_loss=0.2341 train_acc=0.9157 | val_loss=0.3742 val_acc=0.8863
Epoch 8/10 | train_loss=0.2149 train_acc=0.9194 | val_loss=0.3677 val_acc=0.8922
Epoch 9/10 | train_loss=0.2026 train_acc=0.9289 | val_loss=0.3861 val_acc=0.8902
Epoch 10/10 | train_loss=0.2216 train_acc=0.9162 | val_loss=0.3564 val_acc=0.8991


epoch,▁▁▁▁▁▂▂▃▃▃▃▃▃▃▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▆▆▆▆▆▇▇▇██
lr,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
step,▁▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▄▄▅▅▅▅▆▆▆▆▆▆▆▇▇▇▇▇▇▇████
train/acc,▁▂▃▄▄▆▆▇▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇█▇██▇█████▇▇▇▇▇
train/acc_epoch,▁▅▆▆▇▇▇███
train/loss,█▅▅▄▂▃▃▃▂▂▂▂▂▂▂▂▂▂▂▁▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▂▂▂▂▂
train/loss_epoch,█▄▃▂▂▁▁▁▁▁
val/acc,▁▅▄▇▆▆▇▇▇█
val/loss,█▅▅▂▂▃▂▁▂▁
epoch,10
lr,0.001


In [12]:
print("val samples:", len(val_dataset))
print("val batches:", len(val_loader))


val samples: 1011
val batches: 32
